# Demo notebook for Kamodo's Reconstruction: Cartesian Coordinates
This notebook tutorial shows how to fly a constellation of satellites through model data as a virtual reality, focusing on cartesian coordinate examples. Trajectories can be obtained either from the test GDC trajectory file or the flythrough trajectory functions. The GDC test trajectory was designed to be approximately stationary in GSE coordinates. See the Trajectory_Coord_Plots demo notebook for examples on flythrough trajectory options.

In [ ]:
# Get trajectory data from an old GDC trajectory file. 
from kamodo_ccmc.flythrough.Reconstruction_v0 import read_GDC_sattraj, RECON

file_dict = read_GDC_sattraj('C:/Users/rringuet/Kamodo_env/Kamodo/docs/notebooks/KGS_orbit_data.txt')
# coordinates in GDZ spherical: 'UTCtimestamps' in s, 'Longitude' in deg, 'Latitude' in deg, 'Altitude' in km
print(file_dict['UTCtimestamps'].min(), file_dict['UTCtimestamps'].max())  # min and max for later comparison to data

In [ ]:
# Find out what models are available
# The models available through the flythrough are automatically available through the reconstruction function.
from kamodo_ccmc.flythrough import model_wrapper as MW
MW.Choose_Model('')

In [ ]:
# For a given model, find out what time ranges are covered by the data in a given directory.
model = 'GITM'
file_dir = 'D:/GITM/Storm_201303/'  # change to match your machine
times = MW.File_Times(model, file_dir)
times[0].timestamp(), times[1].timestamp()
# This function also automatically performs any data preparation needed.

In [ ]:
# Shift times in example trajectory to match model data time range.
# Reduce computation load by taking every third position
# Some of the trajectory at the end will be automatically excluded later if not covered by the data.
mask = range(0, len(file_dict['UTCtimestamps'])-1, 3)
time_diff = file_dict['UTCtimestamps'][mask].min()-times[0].timestamp()
file_dict['sat_time'] = file_dict['UTCtimestamps'][mask]-time_diff
print(file_dict['sat_time'].min(), file_dict['sat_time'].max(), file_dict['sat_time'].shape, file_dict['UTCtimestamps'].shape)
# Make sure the data you choose covers at least half the time range of the trajectory.

In [ ]:
# Convert coordinates to system desired for reconstruction to take place in.
# See Trajectory_Coords_Plots notebook for more details.
from kamodo_ccmc.flythrough.utils import ConvertCoord
coord_type, coord_grid = 'GEO', 'car'  # choose coordinate system to perform reconstruction in.
c1, c2, c3, units = ConvertCoord(file_dict['sat_time'], file_dict['Longitude'][mask], file_dict['Latitude'][mask], 
                                 file_dict['Altitude'][mask], 'GDZ', 'sph', coord_type, coord_grid)

In [ ]:
# Bring up documentation for the reconstruction functionality.
help(RECON)

In [ ]:
# Choose inputs.
variable_name = 'T_n'  # from chosen files above
recon_dimensions = 'c1c2'  # X vs Y reconstruction for cartesian coordinates
recon_option = 'UnMod_AvgSlice'
# When working in cartesian coordinates, you must be careful with the AvgMod_... and ..._AvgSlice options. In this case, the 
#   default average Z is below the earth's surface (0.666966 R_E), which is not in the domain of the data.
#   If these options are used, you will see simply a ring around the 'earth' on the respective plot
#   showing the data at that Z-slice. Instead, use the Unmod_AvgDSlice option as a starting point. 
#   This means d1 (time resolution in s) and d2 (z resolution in R_E) must be set to indicate the 
#   slice intervals sampled in the averaging.
# Selecting too fine resolution here can lead to memory errors.
d1, d2 = 1800., 0.1

# Set up constellation input:
z_offsets = [0., 0.02, 0.04]  #3 satellites spaced about 2 degrees apart in latitude.
# The offset lists give the offsets of the imaginary satellites with identical trajectories in the 
#  other dimensions (e.g. y and z). Offsets in height are also possible, but not popular.
# Note: Make sure to include the offsets chosen in the next block.

# Note: All offsets are combined to create a number of satellites equal to the multiplied length of each offset array.
#   e.g. lon_offset, lat_offset, time_offset = [-10.,0.,10.], [-10.,0.,10.], [-300.,0.,300.]
#   will yield a constellation of 27 satellites spaced as indicated by the offsets.

# Choose the grid resolution of reconstruction. The finer the resolution, the longer the program takes to run and 
#   the more 'holes' you will see in the reconstructed plot. Physically, these should be set to the instrument's 
#   field of view in the units of the input coordinate system (e.g. degrees for R_E for X, Y, and Z, and seconds for
#   time, etc).
dx, dy = 0.1, 0.1  #Since recon_dimensions='c1c2', dx is resolution in X, and dy is the resolution in Y.

In [ ]:
# Run the reconstruction.
# This process typically takes up to a few minutes, but can take up to ~2 hours depending on the amount of data used,
#  the grid resolution chosen, the reconstruction method chosen, and whether conversion to pressure level is required.
#  As originally set, the process takes about 8 minutes.
# Make sure to include all desired offsets in this block before executing.
recon = RECON(model, variable_name, file_dir, file_dict['sat_time'], c1, c2, c3, coord_type, coord_grid,
              recon_option, recon_dimensions, c3_offsets=z_offsets, dx=dx, dy=dy, d1=d1, d2=d2)

In [ ]:
# The output of the function is a Kamodo object with all of the default features described in documentation.
# T_n is the reconstructed data from the constellation flythrough. T_n_model is the data from the model in the 
#  method chosen. PercentDiff is the percent difference between the two, calculated using 
#  PercentDiff = (T_n_model - T_n)/T_n_model*100.
recon

In [ ]:
# Show the reconstructed plot.
# Note the plots shown are given in the coordinate system associated with the input trajectory.
# Any gaps in the reconstructed plot indicate gaps in the satellite coverage of the plot grid chosen (dx and dy above),
#   and also depend on the constellation arrangement.

# recon.x and recon.y are the x and y grids of the plots.
recon.plot(T_n = dict(Lon=recon.x, Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(T_n = dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the corresponding model data
recon.plot(T_n_model = dict(Lon=recon.x, Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(T_n_model = dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the percent difference between the two. A percent difference of zero is an exact match.
# How well the reconstructed plot matches the model plot not only depends on the constellation arrangement,
#  but also on the reconstruction method chosen. The 'AvgMod_...' options typically result in the better
#  matches, but are not physically representative of what the constellation will 'see' in real data because
#  the two non-reconstructed dimensions are ignored in the input satellite trajectory (e.g. an average value 
#  for both Z and time are used instead of the full range for an X-Y reconstruction). The unmodified 
#  options ('Unmod_...') are thus recommended as the more physical comparison because the full set of input 
#  trajectory values are used.
recon.plot(PercentDiff = dict(Lon=recon.x, Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(PercentDiff = dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Retrieve the percent difference data values and show in a histogram, ignoring NaN values.
# Some extra logic is required to automatically enforce bins of width 2%
import numpy as np
import matplotlib.pyplot as plt

pdiff_data = recon.PercentDiff()
data_min, data_max = np.floor(np.nanmin(pdiff_data)), np.ceil(np.nanmax(pdiff_data))
num_bins = int((data_max-data_min)/2.)
if num_bins<5:
    num_bins=20
hist, edges, patches = plt.hist(np.ravel(pdiff_data), range=(data_min, data_max),bins=num_bins)
plt.xlabel('Percent Difference')
plt.ylabel('Frequency')